In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../'))

import numpy as np
import arrayfire as af
from matplotlib import pyplot as plt

from dg_maxwell import params
from dg_maxwell import msh_parser
from dg_maxwell import advection_2d
from dg_maxwell import global_variables as gvar
from dg_maxwell import utils
from dg_maxwell import wave_equation_2d as w2d
from dg_maxwell import isoparam
from dg_maxwell import lagrange
from dg_maxwell import advection_2d_arbit_mesh as a2d_arbit_mesh

af.set_backend(params.backend)
af.set_device(params.device)

plt.rcParams['figure.figsize']     = 12, 7.5
plt.rcParams['lines.linewidth']    = 1.5
plt.rcParams['font.family']        = 'serif'
plt.rcParams['font.weight']        = 'bold'
plt.rcParams['font.size']          = 20  
plt.rcParams['font.sans-serif']    = 'serif'
plt.rcParams['text.usetex']        = True
plt.rcParams['axes.linewidth']     = 1.5
plt.rcParams['axes.titlesize']     = 'medium'
plt.rcParams['axes.labelsize']     = 'medium'

plt.rcParams['xtick.major.size']   = 8
plt.rcParams['xtick.minor.size']   = 4
plt.rcParams['xtick.major.pad']    = 8
plt.rcParams['xtick.minor.pad']    = 8
plt.rcParams['xtick.color']        = 'k'
plt.rcParams['xtick.labelsize']    = 'medium'
plt.rcParams['xtick.direction']    = 'in'    

plt.rcParams['ytick.major.size']   = 8
plt.rcParams['ytick.minor.size']   = 4
plt.rcParams['ytick.major.pad']    = 8
plt.rcParams['ytick.minor.pad']    = 8
plt.rcParams['ytick.color']        = 'k'
plt.rcParams['ytick.labelsize']    = 'medium'
plt.rcParams['ytick.direction']    = 'in'
plt.rcParams['text.usetex']        = True
plt.rcParams['text.latex.unicode'] = True

gmshtranslator friendly reminder: rules and actions prototypes


def node_condition(tag,x,y,z,physgroups): 
def node_action(tag,x,y,z):
def element_condition(eletag,eletype,physgrp,nodes):
def element_action(eletag,eletype,physgrp,nodes):


In [2]:
params.mesh_file = '../read_and_plot_mesh/mesh/particle_in_rectangle.msh'
advec_var = gvar.advection_variables(params.N_LGL, params.N_quad,
                                     params.x_nodes, params.N_Elements,
                                     params.c, params.total_time, params.wave,
                                     params.c_x, params.c_y, params.courant,
                                     params.mesh_file, params.total_time_2d)

/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


gmshTranslator: Initializing...
gmshTranslator: Mesh has 988 nodes.
gmshTranslator: Mesh has 374 elements.
gmshTranslator: Processed 1366 lines.
gmshTranslator: There are 1 physical groups available: 
gmshTranslator:      > 0
gmshTranslator: Parsing nodes
gmshTranslator: Parsing elements
gmshTranslator: No rules for elements... skipping elements.
gmshTranslator: Parsing nodes
gmshTranslator: No rules for nodes... skipping nodes.
gmshTranslator: Parsing elements
advection_variables __init__ completed


In [3]:
#                                left   right
vertical_boundary_elements_pbc = [[0 ,  184],
                                  [10,  194],
                                  [20,  204],
                                  [30,  214],
                                  [40,  224],
                                  [50,  174],
                                  [60,  164],
                                  [70,  154],
                                  [80,  144],
                                  [90,  134],
                                  [100, 124]]
vertical_boundary_elements_pbc = np.array(vertical_boundary_elements_pbc)

#                                    Top  Bottom
horizontal_boundary_elements_pbc = [[  0, 100],
                                    [  1, 101],
                                    [  2, 102],
                                    [  3, 103],
                                    [  4, 104],
                                    [  5, 105],
                                    [  6, 106],
                                    [  7, 107],
                                    [  8, 108],
                                    [  9, 109],
                                    [225, 110],
                                    [175, 115],
                                    [176, 116],
                                    [177, 117],
                                    [178, 118],
                                    [179, 119],
                                    [180, 120],
                                    [181, 121],
                                    [182, 122],
                                    [183, 123],
                                    [184, 124]]
horizontal_boundary_elements_pbc = np.array(horizontal_boundary_elements_pbc)

In [4]:
interelement_relations = msh_parser.interelement_relations(advec_var.elements)
for horizontal_boundary in horizontal_boundary_elements_pbc:
    print(horizontal_boundary[1], '->', interelement_relations[horizontal_boundary[1]])

100 -> [ -1  -1 101  90]
101 -> [100  -1 102  91]
102 -> [101  -1 103  92]
103 -> [102  -1 104  93]
104 -> [103  -1 105  94]
105 -> [104  -1 106  95]
106 -> [105  -1 107  96]
107 -> [106  -1 108  97]
108 -> [107  -1 109  98]
109 -> [108  -1 110  99]
110 -> [109 111 115  -1]
115 -> [110 125 116  -1]
116 -> [115 126 117  -1]
117 -> [116 127 118  -1]
118 -> [117 128 119  -1]
119 -> [118 129 120  -1]
120 -> [119 130 121  -1]
121 -> [120 131 122  -1]
122 -> gmshTranslator: Ending
[121 132 123  -1]
123 -> [122 133 124  -1]
124 -> [123 134  -1  -1]


In [5]:
# Applying the periodic boundary conditions

for vertical_boundary in vertical_boundary_elements_pbc:
    interelement_relations[vertical_boundary[0], 0] = vertical_boundary[1]
    interelement_relations[vertical_boundary[1], 2] = vertical_boundary[0]

for horizontal_boundary in horizontal_boundary_elements_pbc:
    if horizontal_boundary[1] < 110:
        interelement_relations[horizontal_boundary[0], 3] = horizontal_boundary[1]
        interelement_relations[horizontal_boundary[1], 1] = horizontal_boundary[0]
        
    else:
        interelement_relations[horizontal_boundary[0], 3] = horizontal_boundary[1]
        interelement_relations[horizontal_boundary[1], 3] = horizontal_boundary[0]

In [6]:
def lax_friedrichs_flux(u_n, u_n_plus_1):
    '''
    '''
    lf_flux = None
    
    lf_flux = (w2d.F_x(u_n_plus_1) + w2d.F_x(u_n)) / 2. \
            - params.c_lax * (u_n_plus_1 - u_n) / 2.
    
    return lf_flux

def u_at_edge(u_e_ij, edge_id):
    '''
    Finds the :math:`u` at given edge id for :math:`u_{eij}`.
    
    [Tested]
    '''
    u_e_ij = af.moddims(u_e_ij, params.N_LGL, params.N_LGL)
    
    if edge_id == 0:
        return u_e_ij[:, 0]
    
    elif edge_id == 1:
        return u_e_ij[-1]
    
    elif edge_id == 2:
        return u_e_ij[:, -1]
    
    elif edge_id == 3:
        return u_e_ij[0]
    
    else:
        return
    
    return

# Calculate the Lax-Friedrichs flux for single element

## Testing
- Prelimnary testing seems fine.

In [7]:
element_lf_flux = af.np_to_af_array(np.zeros([advec_var.elements.shape[0],
                                              4, params.N_LGL]))

for element_0_tag in np.arange(advec_var.u_e_ij.shape[1]):
    for element_0_edge_id, element_1_tag in enumerate(interelement_relations[element_0_tag]):
        if element_1_tag != -1:
#             print(element_1_tag, '\t->',
#                   np.where(interelement_relations[element_1_tag] == element_0_tag)[0][0])
            element_1_tag = interelement_relations[element_0_tag][element_0_edge_id]

            element_1_edge_id = np.where(interelement_relations[element_1_tag] == element_0_tag)[0][0]

            u_element_0 = advec_var.u_e_ij[:, element_0_tag]
            u_element_1 = advec_var.u_e_ij[:, element_1_tag]

            u_element_0_at_edge = af.flat(u_at_edge(u_element_0, element_0_edge_id))
            u_element_1_at_edge = af.flat(u_at_edge(u_element_1, element_1_edge_id))
            
            element_lf_flux[element_0_tag, element_0_edge_id] = lax_friedrichs_flux(u_element_0_at_edge,
                                                                                    u_element_1_at_edge)
            
        if element_1_tag == -1:
            print('Element {} Tag at edge {} = -1'.format(element_0_tag, element_0_edge_id))
            u_element_0 = advec_var.u_e_ij[:, element_0_tag]
            
            u_element_0_at_edge = af.flat(u_at_edge(u_element_0, element_0_edge_id))
            u_element_1_at_edge = af.np_to_af_array(np.zeros([params.N_LGL]))
            
            element_lf_flux[element_0_tag,
                            element_0_edge_id] = lax_friedrichs_flux(u_element_0_at_edge,
                                                                     u_element_1_at_edge)
            
print('Done')

Element 59 Tag at edge 2 = -1
Element 114 Tag at edge 1 = -1
Element 165 Tag at edge 0 = -1
Element 229 Tag at edge 1 = -1
Done


# Finding the surface term

In [31]:
# Testing lagrange interlolation
surface_term = af.constant(0., d0 = params.N_LGL ** 2, d1 = advec_var.elements.shape[0])

dx_dxi = 0.1
dy_deta = 0.1

#         p = 0
#         q = 0

# element_tag = 0

for element_tag in np.arange(advec_var.elements.shape[0]):
    for p in np.arange(params.N_LGL):
        for q in np.arange(params.N_LGL):
            
            index = p * params.N_LGL + q
#             print(index)
            
            # Left Edge Integration
            edge_id     = 0

            xi_minus_1    = af.constant(-1., d0 = params.N_LGL, d1 = 1, dtype = af.Dtype.f64)
            Lp_xi_minus_1 = af.flat(utils.polyval_1d(advec_var.lagrange_coeffs[p], xi_minus_1))
            Fxi_minus_1   = af.flat(w2d.F_x(element_lf_flux[element_tag, edge_id]))
            Lq_eta        = af.flat(utils.polyval_1d(advec_var.lagrange_coeffs[q], advec_var.eta_LGL))

            left_edge_integrand = xi_minus_1 * Lp_xi_minus_1 * Fxi_minus_1 * Lq_eta * dy_deta
            left_edge_integrand = lagrange.lagrange_interpolation_u(left_edge_integrand, advec_var)[:, :, 0]

            left_edge_integration = utils.integrate_1d(left_edge_integrand,
                                                       order = 9, scheme = 'gauss')

            # Bottom edge integration
            edge_id     = 1

            eta_minus_1    = af.constant(-1., d0 = params.N_LGL, d1 = 1, dtype = af.Dtype.f64)
            Lq_eta_minus_1 = af.flat(utils.polyval_1d(advec_var.lagrange_coeffs[q], eta_minus_1))
            Feta_minus_1   = af.flat(w2d.F_x(element_lf_flux[element_tag, edge_id]))
            Lp_xi          = af.flat(utils.polyval_1d(advec_var.lagrange_coeffs[p], advec_var.xi_LGL))

            bottom_edge_integrand = eta_minus_1 * Lq_eta_minus_1 * Feta_minus_1 * Lp_xi * dx_dxi
            bottom_edge_integrand = lagrange.lagrange_interpolation_u(bottom_edge_integrand,
                                                                      advec_var)[:, :, 0]

            bottom_edge_integration = utils.integrate_1d(bottom_edge_integrand, order = 9, scheme = 'gauss')


            # Right Edge Integration

            edge_id     = 2

            xi_1    = af.constant(1., d0 = params.N_LGL, d1 = 1, dtype = af.Dtype.f64)
            Lp_xi_1 = af.flat(utils.polyval_1d(advec_var.lagrange_coeffs[p], xi_1))
            Fxi_1   = af.flat(w2d.F_x(element_lf_flux[element_tag, edge_id]))
            Lq_eta  = af.flat(utils.polyval_1d(advec_var.lagrange_coeffs[q], advec_var.eta_LGL))

            right_edge_integrand = xi_1 * Lp_xi_1 * Fxi_1 * Lq_eta * dy_deta
            right_edge_integrand = lagrange.lagrange_interpolation_u(right_edge_integrand,
                                                                     advec_var)[:, :, 0]

            right_edge_integration = utils.integrate_1d(right_edge_integrand,
                                                        order = 9, scheme = 'gauss')

            # Top edge integration

            edge_id     = 3

            eta_1    = af.constant(1., d0 = params.N_LGL, d1 = 1, dtype = af.Dtype.f64)
            Lq_eta_1 = af.flat(utils.polyval_1d(advec_var.lagrange_coeffs[q], eta_1))
            Feta_1   = af.flat(w2d.F_x(element_lf_flux[element_tag, edge_id]))
            Lp_xi    = af.flat(utils.polyval_1d(advec_var.lagrange_coeffs[p], advec_var.xi_LGL))

            top_edge_integrand = eta_1 * Lq_eta_1 * Feta_1 * Lp_xi * dx_dxi
            top_edge_integrand = lagrange.lagrange_interpolation_u(top_edge_integrand,
                                                                   advec_var)[:, :, 0]

            top_edge_integration = utils.integrate_1d(top_edge_integrand, order = 9, scheme = 'gauss')

            surface_term[index, element_tag] = -left_edge_integration - bottom_edge_integration + right_edge_integration + left_edge_integration

print(surface_term[:, 0])

arrayfire.Array()
Type: float

[64 1 1 1]
    0.0021 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0137 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0264 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0372 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0406 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0341 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0000 
   -0.0000 
    0.0205 
   -0.0000 
    0.0000 
   -0.0000 
   -0.0000 
   -0.0000 
   -0.0000 
   -0.0000 
    0.0013 
   -0.0095 
   -0.0039 
    0.0147 
    0.0321 
    0.0334 
    0.0208 
    0.0034 




In [3]:
# surface_term = a2d_arbit_mesh.surface_term_vectorized(advec_var.u_e_ij, advec_var)
a2d_arbit_mesh.time_evolution

Element 59 Tag at edge 2 = -1
Element 114 Tag at edge 1 = -1
Element 165 Tag at edge 0 = -1
Element 229 Tag at edge 1 = -1
Done
-> 0
-> 1
-> 2
-> 3
-> 4
-> 5
-> 6
-> 7
-> 8
-> 9
-> 10
-> 11
-> 12
-> 13
-> 14
-> 15
-> 16
-> 17
-> 18
-> 19
-> 20
-> 21
-> 22
-> 23
-> 24
-> 25
-> 26
-> 27
-> 28
-> 29
-> 30
-> 31
-> 32
-> 33
-> 34
-> 35
-> 36
-> 37
-> 38
-> 39
-> 40
-> 41
-> 42
-> 43
-> 44
-> 45
-> 46
-> 47
-> 48
-> 49
-> 50
-> 51
-> 52
-> 53
-> 54
-> 55
-> 56
-> 57
-> 58
-> 59
-> 60
-> 61
-> 62
-> 63
-> 64
-> 65
-> 66
-> 67
-> 68
-> 69
-> 70
-> 71
-> 72
-> 73
-> 74
-> 75
-> 76
-> 77
-> 78
-> 79
-> 80
-> 81
-> 82
-> 83
-> 84
-> 85
-> 86
-> 87
-> 88
-> 89
-> 90
-> 91
-> 92
-> 93
-> 94
-> 95
-> 96
-> 97
-> 98
-> 99
-> 100
-> 101
-> 102
-> 103
-> 104
-> 105
-> 106
-> 107
-> 108
-> 109
-> 110
-> 111
-> 112
-> 113
-> 114
-> 115
-> 116
-> 117
-> 118
-> 119
-> 120
-> 121
-> 122
-> 123
-> 124
-> 125
-> 126
-> 127
-> 128
-> 129
-> 130
-> 131
-> 132
-> 133
-> 134
-> 135
-> 136
-> 137
-> 138
-> 139
->

arrayfire.Array()
Type: float
Dims: (64, 230)